# [Lab5] SageMaker ML 파이프라인 구축

이 노트북에서는 SageMaker Pipelines를 사용하여 엔드투엔드 ML 워크플로우를 자동화합니다.

## 주요 내용
- 데이터 전처리 파이프라인 단계
- 하이퍼파라미터 튜닝 단계
- 모델 평가 및 조건부 등록
- 파이프라인 실행 및 모니터링

## 1. 환경 설정 및 변수 로드

In [ ]:
# 이전 노트북에서 저장한 변수들 로드
%store -r

print("✅ 저장된 변수들을 로드했습니다.")

In [ ]:
# 필수 라이브러리 임포트
import boto3
import pandas as pd
import sagemaker
import json
import os
from time import gmtime, strftime
from sagemaker.workflow.pipeline_context import PipelineSession
from sagemaker_studio import Project

# 프로젝트 및 파이프라인 설정
project = Project()
role = project.iam_role

# 세션 초기화
sagemaker_session = sagemaker.session.Session()
pipeline_session = PipelineSession()
region = sagemaker_session.boto_region_name
default_bucket = sagemaker_session.default_bucket()

# 파이프라인 설정
pipeline_name = f"bank-marketing-ml-pipeline"
model_package_group_name = f"BankMarketingModelPackageGroup"
base_job_prefix = "bank-marketing"

print(f"✅ 파이프라인 설정 완료")
print(f"   - 파이프라인 이름: {pipeline_name}")
print(f"   - 모델 패키지 그룹: {model_package_group_name}")

In [ ]:
# 파이프라인 스크립트 디렉토리 생성
!mkdir -p pipeline_scripts

## 2. 파이프라인 파라미터 정의

In [ ]:
from sagemaker.workflow.parameters import (
    ParameterInteger,
    ParameterString,
    ParameterFloat
)

# 파이프라인 파라미터 정의
processing_instance_count = ParameterInteger(name="ProcessingInstanceCount", default_value=1)
processing_instance_type = ParameterString(name="ProcessingInstanceType", default_value="ml.m5.xlarge")
training_instance_type = ParameterString(name="TrainingInstanceType", default_value="ml.m5.xlarge")
model_approval_status = ParameterString(name="ModelApprovalStatus", default_value="PendingManualApproval")
auc_score_threshold = ParameterFloat(name="AucScoreThreshold", default_value=0.75)
input_data_uri = ParameterString(
    name="InputDataUri",
    default_value=f"s3://{default_bucket}/bank-additional/bank-additional-full.csv"
)

print("✅ 파이프라인 파라미터 정의 완료")

## 3. 데이터 전처리 단계

In [ ]:
%%writefile "pipeline_scripts/bank_preprocess.py"

import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# 데이터 로드
base_dir = "/opt/ml/processing"
input_path = f"{base_dir}/input/bank-additional-full.csv"
df = pd.read_csv(input_path, sep=';')
print(f"원본 데이터 형태: {df.shape}")

# 타겟 변수 인코딩 (yes=1, no=0)
df['y'] = df['y'].map({'yes': 1, 'no': 0})

# 범주형 변수 원-핫 인코딩
categorical_columns = [
    'job', 'marital', 'education', 'default', 'housing', 'loan',
    'contact', 'month', 'day_of_week', 'poutcome'
]
df_encoded = pd.get_dummies(df, columns=categorical_columns, drop_first=True)

# 특성과 타겟 분리
X = df_encoded.drop('y', axis=1)
y = df_encoded['y']

# 데이터 분할 (70% 훈련, 15% 검증, 15% 테스트)
X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.15, random_state=42, stratify=y)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.176, random_state=42, stratify=y_temp)

# XGBoost 형식으로 저장 (타겟이 첫 번째 열)
train_data = pd.concat([y_train, X_train], axis=1)
val_data = pd.concat([y_val, X_val], axis=1)
test_data = pd.concat([y_test, X_test], axis=1)

# CSV 파일로 저장
train_data.to_csv(f"{base_dir}/train/train.csv", header=False, index=False)
val_data.to_csv(f"{base_dir}/validation/validation.csv", header=False, index=False)
test_data.to_csv(f"{base_dir}/test/test.csv", header=False, index=False)

print("✅ 데이터 전처리 완료")

In [ ]:
# 데이터 전처리 단계 정의
from sagemaker.sklearn.processing import SKLearnProcessor
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker.workflow.steps import ProcessingStep

# SKLearn 프로세서 생성
sklearn_processor = SKLearnProcessor(
    framework_version="1.0-1",
    instance_type=processing_instance_type,
    instance_count=processing_instance_count,
    base_job_name=f"{base_job_prefix}-preprocess",
    role=role,
    sagemaker_session=pipeline_session,
)

# 전처리 단계 인수 정의
processor_args = sklearn_processor.run(
    inputs=[
        ProcessingInput(
            source=input_data_uri,
            destination="/opt/ml/processing/input"
        ),
    ],
    outputs=[
        ProcessingOutput(
            output_name="train",
            source="/opt/ml/processing/train",
            destination=f"s3://{default_bucket}/{base_job_prefix}/data/train"
        ),
        ProcessingOutput(
            output_name="validation",
            source="/opt/ml/processing/validation",
            destination=f"s3://{default_bucket}/{base_job_prefix}/data/validation"
        ),
        ProcessingOutput(
            output_name="test",
            source="/opt/ml/processing/test",
            destination=f"s3://{default_bucket}/{base_job_prefix}/data/test"
        )
    ],
    code="pipeline_scripts/bank_preprocess.py",
)

# 전처리 단계 생성
step_process = ProcessingStep(
    name="BankMarketingDataProcess",
    step_args=processor_args
)

print("✅ 데이터 전처리 단계 정의 완료")

## 4. 하이퍼파라미터 튜닝 단계

In [ ]:
from sagemaker.estimator import Estimator
from sagemaker.inputs import TrainingInput
from sagemaker.tuner import (
    IntegerParameter,
    CategoricalParameter,
    ContinuousParameter,
    HyperparameterTuner,
)
from sagemaker.workflow.steps import TuningStep

# XGBoost 컨테이너 이미지 URI
image_uri = sagemaker.image_uris.retrieve(
    framework="xgboost",
    region=region,
    version="1.7-1",
    py_version="py3",
    instance_type=training_instance_type,
)

# 고정 하이퍼파라미터
fixed_hyperparameters = {
    "eval_metric": "auc",
    "objective": "binary:logistic",
    "num_round": "100",
    "early_stopping_rounds": "10",
}

# XGBoost Estimator 생성
xgb_estimator = Estimator(
    image_uri=image_uri,
    instance_type=training_instance_type,
    instance_count=1,
    hyperparameters=fixed_hyperparameters,
    output_path=f"s3://{default_bucket}/{base_job_prefix}/models",
    base_job_name=f"{base_job_prefix}-train",
    sagemaker_session=pipeline_session,
    role=role,
)

# 하이퍼파라미터 튜닝 범위 정의
hyperparameter_ranges = {
    "eta": ContinuousParameter(0.01, 0.3),           # 학습률
    "max_depth": IntegerParameter(3, 10),            # 트리 최대 깊이
    "min_child_weight": ContinuousParameter(1, 10),  # 최소 자식 가중치
    "subsample": ContinuousParameter(0.7, 1.0),      # 서브샘플링 비율
    "alpha": ContinuousParameter(0, 2),              # L1 정규화
}

# 하이퍼파라미터 튜너 생성
tuner = HyperparameterTuner(
    xgb_estimator,
    "validation:auc",
    hyperparameter_ranges,
    max_jobs=4,        # 최대 튜닝 작업 수
    max_parallel_jobs=2,  # 병렬 실행 작업 수
    objective_type="Maximize",
    base_tuning_job_name=f"{base_job_prefix}-tuning"
)

# 하이퍼파라미터 튜닝 단계 생성
hpo_args = tuner.fit(
    inputs={
        "train": TrainingInput(
            s3_data=step_process.properties.ProcessingOutputConfig.Outputs["train"].S3Output.S3Uri,
            content_type="text/csv",
        ),
        "validation": TrainingInput(
            s3_data=step_process.properties.ProcessingOutputConfig.Outputs["validation"].S3Output.S3Uri,
            content_type="text/csv",
        ),
    }
)

step_tuning = TuningStep(
    name="BankMarketingHyperParameterTuning",
    step_args=hpo_args,
)

print("✅ 하이퍼파라미터 튜닝 단계 생성 완료")

## 5. 모델 평가 단계

In [ ]:
%%writefile "pipeline_scripts/bank_evaluate.py"

import json
import pathlib
import pickle
import tarfile
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.metrics import roc_auc_score, accuracy_score

# 모델 로드
model_path = "/opt/ml/processing/model/model.tar.gz"
with tarfile.open(model_path) as tar:
    tar.extractall(path=".")
model = pickle.load(open("xgboost-model", "rb"))
print("✅ 모델 로드 완료")

# 테스트 데이터 로드
test_path = "/opt/ml/processing/test/test.csv"
df = pd.read_csv(test_path, header=None)
y_test = df.iloc[:, 0].to_numpy()
X_test = df.iloc[:, 1:].values
print(f"테스트 데이터 형태: {X_test.shape}")

# XGBoost DMatrix 생성
dtest = xgb.DMatrix(X_test)

# 예측 수행
predictions_proba = model.predict(dtest)
predictions_binary = (predictions_proba > 0.5).astype(int)

# 성능 메트릭 계산
auc_score = roc_auc_score(y_test, predictions_proba)
accuracy = accuracy_score(y_test, predictions_binary)

print(f"\n📊 평가 결과:")
print(f"   - AUC: {auc_score:.4f}")
print(f"   - 정확도: {accuracy:.4f}")

# 평가 리포트 생성
report_dict = {
    "classification_metrics": {
        "auc_score": {
            "value": float(auc_score),
            "standard_deviation": 0.0
        },
        "accuracy": {
            "value": float(accuracy),
            "standard_deviation": 0.0
        }
    }
}

# 평가 결과 저장
output_dir = "/opt/ml/processing/evaluation"
pathlib.Path(output_dir).mkdir(parents=True, exist_ok=True)
evaluation_path = f"{output_dir}/evaluation.json"
with open(evaluation_path, "w") as f:
    f.write(json.dumps(report_dict, indent=2))

print(f"✅ 평가 결과 저장 완료")

In [ ]:
# 모델 평가 단계 정의
from sagemaker.processing import ScriptProcessor
from sagemaker.workflow.properties import PropertyFile

# 스크립트 프로세서 생성
script_eval = ScriptProcessor(
    image_uri=image_uri,
    command=["python3"],
    instance_type=processing_instance_type,
    instance_count=1,
    base_job_name=f"{base_job_prefix}-eval",
    role=role,
    sagemaker_session=pipeline_session,
)

# 평가 단계 인수 정의
eval_args = script_eval.run(
    inputs=[
        ProcessingInput(
            source=step_tuning.get_top_model_s3_uri(
                top_k=0, 
                s3_bucket=default_bucket, 
                prefix=f"{base_job_prefix}/models"
            ),
            destination="/opt/ml/processing/model"
        ),
        ProcessingInput(
            source=step_process.properties.ProcessingOutputConfig.Outputs["test"].S3Output.S3Uri,
            destination="/opt/ml/processing/test"
        )
    ],
    outputs=[
        ProcessingOutput(
            output_name="evaluation",
            source="/opt/ml/processing/evaluation",
            destination=f"s3://{default_bucket}/{base_job_prefix}/evaluation"
        ),
    ],
    code="pipeline_scripts/bank_evaluate.py",
)

# 평가 리포트 속성 파일
evaluation_report = PropertyFile(
    name="BankMarketingEvaluationReport",
    output_name="evaluation",
    path="evaluation.json"
)

# 평가 단계 생성
step_eval = ProcessingStep(
    name="BankMarketingEvalModel",
    step_args=eval_args,
    property_files=[evaluation_report],
)

print("✅ 모델 평가 단계 생성 완료")

## 6. 모델 등록 및 조건부 단계

In [ ]:
from sagemaker import Model
from sagemaker.workflow.model_step import ModelStep
from sagemaker.model_metrics import MetricsSource, ModelMetrics

# 모델 생성
model = Model(
    image_uri=image_uri,
    model_data=step_tuning.get_top_model_s3_uri(
        top_k=0, 
        s3_bucket=default_bucket, 
        prefix=f"{base_job_prefix}/models"
    ),
    sagemaker_session=pipeline_session,
    role=role,
)

# 모델 메트릭 정의
model_metrics = ModelMetrics(
    model_statistics=MetricsSource(
        s3_uri="{}/evaluation.json".format(
            step_eval.arguments["ProcessingOutputConfig"]["Outputs"][0]["S3Output"]["S3Uri"]
        ),
        content_type="application/json",
    )
)

# 모델 등록 인수 정의
register_args = model.register(
    content_types=["text/csv"],
    response_types=["text/csv"],
    inference_instances=["ml.t2.medium", "ml.m5.large"],
    transform_instances=["ml.m5.xlarge"],
    model_package_group_name=model_package_group_name,
    approval_status=model_approval_status,
    model_metrics=model_metrics,
    description="은행 마케팅 캠페인 정기예금 가입 예측 모델",
)

# 모델 등록 단계 생성
step_register = ModelStep(
    name="BankMarketingRegisterModel",
    step_args=register_args
)

print("✅ 모델 등록 단계 생성 완료")

In [ ]:
from sagemaker.workflow.conditions import ConditionGreaterThan
from sagemaker.workflow.condition_step import ConditionStep
from sagemaker.workflow.functions import JsonGet

# AUC 점수 조건 정의
cond_gte = ConditionGreaterThan(
    left=JsonGet(
        step_name=step_eval.name,
        property_file=evaluation_report,
        json_path="classification_metrics.auc_score.value",
    ),
    right=auc_score_threshold,
)

# 조건부 단계 생성
step_cond = ConditionStep(
    name="CheckAUCScoreBankMarketingEvaluation",
    conditions=[cond_gte],
    if_steps=[step_register],  # AUC가 임계값보다 높으면 모델 등록
    else_steps=[],             # 그렇지 않으면 아무것도 하지 않음
)

print("✅ 조건부 단계 생성 완료")

In [ ]:
from sagemaker.workflow.pipeline import Pipeline

# 파이프라인 생성
pipeline = Pipeline(
    name=pipeline_name,
    parameters=[
        processing_instance_count,
        processing_instance_type,
        training_instance_type,
        model_approval_status,
        auc_score_threshold,
        input_data_uri,
    ],
    steps=[
        step_process,    # 1. 데이터 전처리
        step_tuning,     # 2. 하이퍼파라미터 튜닝
        step_eval,       # 3. 모델 평가
        step_cond,       # 4. 조건부 모델 등록
    ],
    sagemaker_session=pipeline_session,
)

print("✅ 파이프라인 생성 완료")

In [ ]:
# 파이프라인 정의 확인
definition = json.loads(pipeline.definition())

print("📋 파이프라인 정의 요약:")
print(f"   - 버전: {definition['Version']}")
print(f"   - 파라미터 수: {len(definition['Parameters'])}")
print(f"   - 단계 수: {len(definition['Steps'])}")

In [ ]:
# 파이프라인 등록/업데이트
pipeline.upsert(role_arn=role)
print(f"✅ 파이프라인 등록 완료: {pipeline_name}")

In [ ]:
# 파이프라인 실행
execution = pipeline.start(
    parameters={
        "ProcessingInstanceType": "ml.m5.xlarge",
        "TrainingInstanceType": "ml.m5.xlarge",
        "AucScoreThreshold": 0.75,
        "ModelApprovalStatus": "PendingManualApproval"
    }
)

print(f"✅ 파이프라인 실행 시작됨")
print(f"   - 실행 ARN: {execution.arn}")

## 8. 파이프라인 모니터링

In [ ]:
# 파이프라인 실행 상태 확인
if 'execution' in locals():
    status = execution.describe()
    
    print(f"\n🔍 실행 정보:")
    print(f"   - 상태: {status['PipelineExecutionStatus']}")
    print(f"   - 시작 시간: {status['CreationTime']}")
    
    # 단계별 상태 확인
    steps = execution.list_steps()
    
    print(f"\n📋 단계별 상태:")
    for step in steps:
        step_name = step['StepName']
        step_status = step['StepStatus']
        print(f"   - {step_name}: {step_status}")
else:
    print("⚠️ 실행 중인 파이프라인이 없습니다.")

## 9. 파이프라인 결과 요약

In [ ]:
# 파이프라인 구축 결과 요약
print("📋 SageMaker ML 파이프라인 구축 완료!")
print("=" * 50)

print(f"🏗️ 구축된 파이프라인:")
print(f"   - 이름: {pipeline_name}")
print(f"   - 모델 패키지 그룹: {model_package_group_name}")

print(f"\n🔧 파이프라인 구성 요소:")
print(f"   1. 데이터 전처리 (SKLearnProcessor)")
print(f"   2. 하이퍼파라미터 튜닝 (XGBoost)")
print(f"   3. 모델 평가 (테스트 데이터)")
print(f"   4. 조건부 모델 등록 (AUC > {auc_score_threshold.default_value})")

print(f"\n⚙️ 파라미터 설정:")
print(f"   - 처리 인스턴스: {processing_instance_type.default_value}")
print(f"   - 훈련 인스턴스: {training_instance_type.default_value}")
print(f"   - AUC 임계값: {auc_score_threshold.default_value}")

print("\n" + "=" * 50)
print("🎉 ML 파이프라인 구축 완료!")